# Freespace
- import data [x]
- set up representation yaw, pitch, depth [x]
- solver resolution - nearby points creates boundary? [x]
- visualize on example [x]


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from vis.deprecated_vis import *
from data.PATHS import TMP_VIS_PATH
from data.NSF_data import NSF_dataset
from data.range_image import range_image_coords, create_depth_img

dataset = NSF_dataset()
data = next(dataset.__iter__())

torch.use_deterministic_algorithms(mode=True, warn_only=False)  # this ...
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pc1, pc2, gt_flow = data

pc1 = pc1.to(device)
pc2 = pc2.to(device)
gt_flow = gt_flow.to(device)
est_flow = torch.randn(pc1.shape, device=device, requires_grad=True)

In [2]:
from pytorch3d.ops.knn import knn_points

def calculate_polar_coords(pc):

    calc_depth = torch.norm(pc[:,:3], dim=1)
    yaw = - torch.arctan2(pc[:, 1], pc[:, 0])
    pitch = torch.arcsin(pc[:, 2] / (calc_depth + 1e-8))

    return yaw, pitch, calc_depth


def yaw_pitch_depth_to_xyz(yaw, pitch, depth):
    # generated, maybe shifted
    # Convert degrees to radians
    # yaw = math.radians(yaw)
    # pitch = math.radians(pitch)

    # correct [x]
    # Calculate XYZ coordinates
    x = depth * torch.cos(yaw) * torch.cos(pitch)
    y = depth * torch.sin(yaw) * torch.cos(pitch)
    z = depth * torch.sin(pitch)

    xyz = torch.stack([x, y, z]).T#[..., [2, 0, 1]]
    xyz[:, 1] = - xyz[:, 1]

    return xyz



# Representation

# Create freespace label

In [5]:
t_z1 = 1.640420
t_z2 = 1.565252
t_z_mean = (t_z1 + t_z2) / 2
# todo tohle asi zpusobuje error v rekonstrukci, dva body maji diky jinemu souradnemu systemu stejny pitch a yaw, a pak jsou za sebou a jeden ma depth malou a druhy velkou. proto se do point cloudu promitne jinak. - Ale proc range image je jiny, kdyz vstupy jsou stejne?!?

# resolution based on the data?
# todo pouzit puvodni point cloud []
fov_up= 25
fov_down = - 25
proj_H = 50 #?
proj_W = 2048
max_range = 70

to_project_pc2 = pc2[0].clone()
to_project_pc2[:, 2] -= t_z_mean


flow_pc = pc2[0].clone() # + torch.randn_like(pc2[0], device=pc2.device) * 0.1
flow_pc[:, 2] -= t_z_mean



NameError: name 'range_depth' is not defined

In [18]:
# VAChamfer()
# FlowSmoothness

class FlowFreespace(torch.nn.Module):

    def __init__(self, pc2, fov_up, fov_down, proj_H, proj_W, max_range, margin=0.001):
        super().__init__()
        # todo dataset lidar pose
        # KNN smoothness vis-aware
        # todo option of "pushing" points to the freespace
        self.pc2 = pc2
        self.fov_up = fov_up
        self.fov_down = fov_down
        self.proj_H = proj_H
        self.proj_W = proj_W
        self.max_range = max_range
        self.margin = margin

        pc2_depth, idx_w, idx_h, inside_range_img = range_image_coords(to_project_pc2, fov_up, fov_down, proj_H, proj_W)
        self.range_depth = create_depth_img(pc2_depth, idx_w, idx_h, proj_H, proj_W, inside_range_img)

    def forward(self, pc1, est_flow):
        '''

        Args:
            pc1:
            est_flow:

        Returns:
        mask whether the deformed point cloud is in freespace visibility area
        '''
        # dynamic
        # create deformed pc1
        flow_pc = (pc1 + est_flow)[0]

        flow_depth, flow_w, flow_h, flow_inside = range_image_coords(flow_pc, self.fov_up, self.fov_down, self.proj_H, self.proj_W)

        # use it only for flow inside the image
        masked_pc2_depth = self.range_depth[flow_h[flow_inside], flow_w[flow_inside]]
        compared_depth = masked_pc2_depth - flow_depth[flow_inside]

        # compared_depth
        # if flow point before the visible point from pc2, then it is in freespace
        # margin is just little number to not push points already close to visible point
        flow_in_freespace = compared_depth > 0 + self.margin

        return flow_in_freespace

FlowFreespace(to_project_pc2, fov_up, fov_down, proj_H, proj_W, max_range)(pc1, est_flow)

tensor([ True, False, False,  ...,  True, False, False], device='cuda:0')